<a href="https://colab.research.google.com/github/ttobaegi/ML_Python/blob/master/EDA_Ecommerce2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# drive를 colab에 연결해서 데이터 로딩하기
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd     # data preprocessing
import numpy as np

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

plt.style.use('seaborn-whitegrid')

# 파일 불러오기

In [ ]:
master=pd.read_csv('/content/drive/MyDrive/Project_2021/master1.csv', sep=',')
customer=pd.read_csv('/content/drive/MyDrive/Project_2021/Custom.csv', sep=',')
search1=pd.read_csv('/content/drive/MyDrive/Project_2021/Search1.csv', sep=',')
search2=pd.read_csv('/content/drive/MyDrive/Project_2021/Search2.csv', sep=',')
session=pd.read_csv('/content/drive/MyDrive/Project_2021/Session.csv', sep=',')
product=pd.read_csv('/content/drive/MyDrive/Project_2021/Product.csv', sep=',')
indicator1=pd.read_csv('/content/drive/MyDrive/Project_2021/indicator1.csv', sep=',')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# PreProcessing
1. 컬럼명 소문자로 변경

In [ ]:
#컬럼명 소문자로 변경
session_col = session.columns
search1_col = search1.columns
search2_col = search2.columns
customer_col = customer.columns
master_col = master.columns
product_col = product.columns

cols = (session_col, search1_col, search2_col, customer_col, master_col, product_col)
df = (session, search1, search2, customer, master, product)
for i in range(0,6):
  c_lower = []
  for ii in cols[i] :
    c_lower.append(ii.lower())
  df[i].columns=c_lower

In [ ]:
for i in range(0,6):
  print(df[i].columns)

Index(['clnt_id', 'sess_id', 'sess_seq', 'sess_dt', 'tot_pag_view_ct',
       'tot_sess_hr_v', 'dvc_ctg_nm', 'zon_nm', 'city_nm'],
      dtype='object')
Index(['clnt_id', 'sess_id', 'kwd_nm', 'search_cnt'], dtype='object')
Index(['sess_dt', 'kwd_nm', 'search_cnt'], dtype='object')
Index(['clnt_id', 'clnt_gender', 'clnt_age'], dtype='object')
Index(['pd_c', 'pd_nm', 'clac1_nm', 'clac2_nm', 'clac3_nm'], dtype='object')
Index(['clnt_id', 'sess_id', 'hits_seq', 'pd_c', 'pd_add_nm', 'pd_bra_nm',
       'pd_buy_am', 'pd_buy_ct'],
      dtype='object')


In [ ]:
# any() > series df 모두 적용 가능
# checks whether any value in the caller object is not 0 and returns True for that
session.isnull().any()
# tot_sesshr_v, tot_pag_view_ct 에 null값 존재

clnt_id            False
sess_id            False
sess_seq           False
sess_dt            False
tot_pag_view_ct     True
tot_sess_hr_v       True
dvc_ctg_nm         False
zon_nm             False
city_nm            False
dtype: bool

In [ ]:
customer.head(1)

,clnt_id,clnt_gender,clnt_age
0,4830726,F,30


In [ ]:
master.head(1)  # Product 상세 정보 포함 (대분류 중분류 소분류 상품이름)

,pd_c,pd_nm,clac1_nm,clac2_nm,clac3_nm
0,64382,언더아머 남성 UA HG 아머 모크 LS 1289559-001 - 블랙 / MD[95],스포츠패션,남성일반스포츠의류,남성스포츠티셔츠


In [ ]:
print(product.info())
print(master.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5024906 entries, 0 to 5024905
Data columns (total 8 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   clnt_id    int64 
 1   sess_id    int64 
 2   hits_seq   int64 
 3   pd_c       int64 
 4   pd_add_nm  object
 5   pd_bra_nm  object
 6   pd_buy_am  object
 7   pd_buy_ct  object
dtypes: int64(4), object(4)
memory usage: 306.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 847634 entries, 0 to 847633
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   pd_c      847634 non-null  int64 
 1   pd_nm     847634 non-null  object
 2   clac1_nm  847634 non-null  object
 3   clac2_nm  847634 non-null  object
 4   clac3_nm  847634 non-null  object
dtypes: int64(1), object(4)
memory usage: 32.3+ MB
None


In [ ]:
product.head(1)

,clnt_id,sess_id,hits_seq,pd_c,pd_add_nm,pd_bra_nm,pd_buy_am,pd_buy_ct
0,4139680,7605037,12,642112,색상:워터멜론,[바비브라운],"39,000",1


In [ ]:
# df[df['col'] 조건] 해당하는 열만 추출
product[product['pd_buy_ct']==0]        

,clnt_id,sess_id,hits_seq,pd_c,pd_add_nm,pd_bra_nm,pd_buy_am,pd_buy_ct


- product 테이블은 구매내역만 포함하고 있음 
- Orders table 생성(Products, sessions, customer 정보)

In [ ]:
# left/right join : pd.merge(tables, left_on='key', right_on='key', how='left'|'right')
# drop columns : df.drop(['col1','col2',...], axis=1)
# rename column

orders = pd.merge(product, session, left_on='sess_id', right_on='sess_id', how='left').drop(['pd_bra_nm', 'clnt_id_y'], axis=1)
orders.head(2)

,clnt_id_x,sess_id,hits_seq,pd_c,pd_add_nm,pd_buy_am,pd_buy_ct,sess_seq,sess_dt,tot_pag_view_ct,tot_sess_hr_v,dvc_ctg_nm,zon_nm,city_nm
0,4139680,7605037,12,642112,색상:워터멜론,"39,000",1,1,20180526,21.0,902,desktop,Seoul,Seoul
1,4140076,10189797,13,570603,색상:BLK0_(BLK0)BLACK|사이즈:120 / 2개,"39,000",2,172,20180413,18.0,113,mobile,Gyeonggi-do,Icheon-si


In [ ]:
orders=pd.merge(orders, master, left_on="pd_c", right_on="pd_c", how='left').drop(['pd_add_nm'], axis=1).rename(columns={'clnt_id_x':'clnt_id'}, inplace=True) 
orders.head(2)

,clnt_id_x,sess_id,hits_seq,pd_c,pd_buy_am,pd_buy_ct,sess_seq,sess_dt,tot_pag_view_ct,tot_sess_hr_v,dvc_ctg_nm,zon_nm,city_nm,pd_nm,clac1_nm,clac2_nm,clac3_nm
0,4139680,7605037,12,642112,"39,000",1,1,20180526,21.0,902,desktop,Seoul,Seoul,크러쉬드 립칼라 - 플럼,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터
1,4140076,10189797,13,570603,"39,000",2,172,20180413,18.0,113,mobile,Gyeonggi-do,Icheon-si,[데상트] 스포츠베이직 빅로고 티셔츠 - DMGR_DMELANGE / 115,스포츠패션,남성일반스포츠의류,남성스포츠티셔츠


In [ ]:
orders=pd.merge(orders, customer, left_on="clnt_id", right_on="clnt_id", how='left')
orders.head(2)

,clnt_id,sess_id,hits_seq,pd_c,pd_buy_am,pd_buy_ct,sess_seq,sess_dt,tot_pag_view_ct,tot_sess_hr_v,dvc_ctg_nm,zon_nm,city_nm,pd_nm,clac1_nm,clac2_nm,clac3_nm,clnt_gender,clnt_age
0,4139680,7605037,12,642112,"39,000",1,1,20180526,21.0,902,desktop,Seoul,Seoul,크러쉬드 립칼라 - 플럼,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터,NaN,NaN
1,4140076,10189797,13,570603,"39,000",2,172,20180413,18.0,113,mobile,Gyeonggi-do,Icheon-si,[데상트] 스포츠베이직 빅로고 티셔츠 - DMGR_DMELANGE / 115,스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,F,40.0


- clnt gender / age 정보비회원으로 구매한경우 NaN

In [ ]:
orders.head(2)

,clnt_id,sess_id,hits_seq,pd_c,pd_buy_am,pd_buy_ct,sess_seq,sess_dt,tot_pag_view_ct,tot_sess_hr_v,dvc_ctg_nm,zon_nm,city_nm,pd_nm,clac1_nm,clac2_nm,clac3_nm,clnt_gender,clnt_age
0,4139680,7605037,12,642112,"39,000",1,1,20180526,21.0,902,desktop,Seoul,Seoul,크러쉬드 립칼라 - 플럼,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터,NaN,NaN
1,4140076,10189797,13,570603,"39,000",2,172,20180413,18.0,113,mobile,Gyeonggi-do,Icheon-si,[데상트] 스포츠베이직 빅로고 티셔츠 - DMGR_DMELANGE / 115,스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,F,40.0


In [ ]:
orders.loc[0,'clnt_age']

nan

In [ ]:
orders[orders['clnt_age']=='nan']

,clnt_id,sess_id,hits_seq,pd_c,pd_buy_am,pd_buy_ct,sess_seq,sess_dt,tot_pag_view_ct,tot_sess_hr_v,dvc_ctg_nm,zon_nm,city_nm,pd_nm,clac1_nm,clac2_nm,clac3_nm,clnt_gender,clnt_age


# EDA Tasks


1. Check the target variable: Amount of attrition.

2. Check the demographic variables: CUSTOMERRELATED
  * Age vs attrition.
  * Gender vs churn.
  * Number of dependents vs churn.
  * Education level vs churn.
  * Marital status vs churn.
  * Income category vs churn.

3. Check the product variables:
Type of card vs churn.
Relationship with the bank vs churn.
Number of products vs churn.
Inactive months vs churn.
Number of contacts vs churn.
Credit Limit vs churn.
Total resolving balance vs churn.
Openness To Buy Credit Line vs churn.
Transaction Amount Change vs churn.
Transaction Count Change vs churn.
Average Card Utilization Ratio vs churn.

In [ ]:
master.shape

(847634, 5)

In [ ]:
master.head(1)

,pd_c,pd_nm,clac1_nm,clac2_nm,clac3_nm
0,64382,언더아머 남성 UA HG 아머 모크 LS 1289559-001 - 블랙 / MD[95],스포츠패션,남성일반스포츠의류,남성스포츠티셔츠


In [ ]:
customer.head(1)

,CLNT_ID,CLNT_GENDER,CLNT_AGE
0,4830726,F,30


In [ ]:
search1.head(1)

,CLNT_ID,SESS_ID,KWD_NM,SEARCH_CNT
0,5607714,7112876,빌리프 아이크림,6


In [ ]:
search2.head(1)

,SESS_DT,KWD_NM,SEARCH_CNT
0,20180407,닥스원피스,8


In [ ]:
session.head(1)

,clnt_id,sess_id,sess_seq,sess_dt,tot_pag_view_ct,tot_sess_hr_v,dvc_ctg_nm,zon_nm,city_nm
0,5873599,8641867,9,20180509,82.0,"1,890",mobile,Daejeon,Daejeon


In [ ]:
product.head(1)

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT
0,4139680,7605037,12,642112,색상:워터멜론,[바비브라운],"39,000",1


In [ ]:
product.head(1)

,clnt_id,sess_id,hits_seq,pd_c,pd_add_nm,pd_bra_nm,pd_buy_am,pd_buy_ct
0,4139680,7605037,12,642112,색상:워터멜론,[바비브라운],"39,000",1


In [ ]:
session.shape

(2712907, 9)

In [ ]:
session.columns

Index(['clnt_id', 'sess_id', 'sess_seq', 'sess_dt', 'tot_pag_view_ct',
       'tot_sess_hr_v', 'dvc_ctg_nm', 'zon_nm', 'city_nm'],
      dtype='object')

In [ ]:
session.info() 
# date 변수 변환 필요 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2712907 entries, 0 to 2712906
Data columns (total 9 columns):
 #   Column           Dtype  
---  ------           -----  
 0   clnt_id          int64  
 1   sess_id          int64  
 2   sess_seq         int64  
 3   sess_dt          int64  
 4   tot_pag_view_ct  float64
 5   tot_sess_hr_v    object 
 6   dvc_ctg_nm       object 
 7   zon_nm           object 
 8   city_nm          object 
dtypes: float64(1), int64(4), object(4)
memory usage: 186.3+ MB


In [ ]:
pd.set_option('float_format','{:f}'.format)
session.describe()
# 201804-201809 6개월 session 데이터 

,clnt_id,sess_id,sess_seq,sess_dt,tot_pag_view_ct
count,2712907.000000,2712907.000000,2712907.000000,2712907.000000,2712633.000000
mean,3318866.710368,5796890.738461,178.130644,20180648.425562,85.167676
std,1914523.994231,3095032.611309,297.158311,166.288816,87.508775
min,8.000000,26.000000,1.000000,20180401.000000,1.000000
25%,1656959.000000,3220733.000000,14.000000,20180513.000000,29.000000
50%,3318627.000000,5967623.000000,59.000000,20180622.000000,55.000000
75%,4984705.500000,8421878.000000,211.000000,20180807.000000,108.000000
max,6592554.000000,10977941.000000,7020.000000,20180930.000000,499.000000


In [ ]:
# date변수 str 변환 > apply lambda
session['sess_dt']=session['sess_dt'].apply(lambda s : str(s)[0:10])
session.describe()

,clnt_id,sess_id,sess_seq,tot_pag_view_ct
count,2712907.000000,2712907.000000,2712907.000000,2712633.000000
mean,3318866.710368,5796890.738461,178.130644,85.167676
std,1914523.994231,3095032.611309,297.158311,87.508775
min,8.000000,26.000000,1.000000,1.000000
25%,1656959.000000,3220733.000000,14.000000,29.000000
50%,3318627.000000,5967623.000000,59.000000,55.000000
75%,4984705.500000,8421878.000000,211.000000,108.000000
max,6592554.000000,10977941.000000,7020.000000,499.000000


##2. Check the demographic variables: CUSTOMERRELATED
  * Age vs attrition.
  * Gender vs churn.
  * Number of dependents vs churn.
  * Education level vs churn.
  * Marital status vs churn.
  * Income category vs churn.

***How many customer have churned ?***

***How many Customers visit the site during the period ?***

In [ ]:
# session.iloc[:,0].unique
visitors = session['clnt_id'].nunique()
print('visitors:{}'.format(visitors))

visitors:922737


***Visitors daily Trend : Does traffic flunctuate by date?***

In [ ]:
session.drop_duplicates().groupby(['sess_dt'])['clnt_id'].agg(['count'])

,count
sess_dt,
20180401,19274
20180402,19775
20180403,18939
20180404,17879
20180405,16757
...,...
20180926,13361
20180927,13333
20180928,12667


In [ ]:
session.drop_duplicates().groupby(['SESS_DT'])['CLNT_ID'].agg(['count']).sort_values(by=['SESS_DT'],ascending=True)

,count
SESS_DT,
20180401,19274
20180402,19775
20180403,18939
20180404,17879
20180405,16757
...,...
20180926,13361
20180927,13333
20180928,12667


In [ ]:
visitors_by_date = session.drop_duplicates().groupby(['SESS_DT'])['CLNT_ID'].agg(['count']).sort_values(by=['SESS_DT'],ascending=True)

In [ ]:
 from datetime import datetime

In [ ]:
pd.Series(visitors_by_date.index.values).apply(lambda s: datetime.strptime(s,'%Y%m%d').date())

0      2018-04-01
1      2018-04-02
2      2018-04-03
3      2018-04-04
4      2018-04-05
          ...    
178    2018-09-26
179    2018-09-27
180    2018-09-28
181    2018-09-29
182    2018-09-30
Length: 183, dtype: object

In [ ]:
visitors_by_date['count']

SESS_DT
20180401    19274
20180402    19775
20180403    18939
20180404    17879
20180405    16757
            ...  
20180926    13361
20180927    13333
20180928    12667
20180929    12108
20180930    14106
Name: count, Length: 183, dtype: int64

In [ ]:
x = pd.Series(visitors_by_date.index.values).apply(lambda s: datetime.strptime(s,'%Y%m%d').date())
y = visitors_by_date['count']
plt.rcParams['figure.figsize']=(20,8)
plt.plot(x,y)
plt.show()

NameError: ignored

*Feature Engineering*